In [1]:
import pandas as pd

In [10]:
trainBodies = pd.read_csv('fncData/train_bodies.csv')
trainStances = pd.read_csv('fncData/train_stances.csv')
trainStancesRandom = pd.read_csv('fncData/train_stances.random.csv')

print "Train Bodies"
trainBodies.head(20)
trainBodies.get_value(1)

Train Bodies


TypeError: get_value() takes at least 3 arguments (2 given)

In [8]:
print "Train Stances"
trainStances.head(20)

trainStances.addColimn(stance === unrelated)

Train Stances


,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree
5,'Nasa Confirms Earth Will Experience 6 Days of...,154,agree
6,Accused Boston Marathon Bomber Severely Injure...,962,unrelated
7,Identity of ISIS terrorist known as 'Jihadi Jo...,2033,unrelated
8,Banksy 'Arrested & Real Identity Revealed' Is ...,1739,agree
9,British Aid Worker Confirmed Murdered By ISIS,882,unrelated


In [7]:
print "Train Stances Random"
trainStancesRandom.head(20)

Train Stances Random


,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,discuss
1,Hundreds of Palestinians flee floods in Gaza a...,158,discuss
2,"Christian Bale passes on role of Steve Jobs, a...",137,disagree
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,disagree
4,Spider burrowed through tourist's stomach and ...,1923,agree
5,'Nasa Confirms Earth Will Experience 6 Days of...,154,discuss
6,Accused Boston Marathon Bomber Severely Injure...,962,unrelated
7,Identity of ISIS terrorist known as 'Jihadi Jo...,2033,agree
8,Banksy 'Arrested & Real Identity Revealed' Is ...,1739,agree
9,British Aid Worker Confirmed Murdered By ISIS,882,agree
